# Practical Docker: Part 1
---------------------------------------------------------
## Containers: Build Once, Run Anywhere


### *IT/EAD/Flex Solutions Penang*
*2019'WW40* 

*by VS* (vui.seng.chua@intel.com)

>  "The Good Old Days"
>
>> # ¯\\_(ツ)_/¯
> # It works on my machine!!

# Convention in this presentation

```Commands will be highlighted like this```

These are commands for you try out at your end when following through the slides.

.................................................................................................................................................................................

```
Traces in the terminal...
.
.
.
```

Unhighlighted block with monospace font is the example output of commands. You should expect to see similar output at your end.

.................................................................................................................................................................................

##### Quiz: 
Checkpoint quizzes for self-assessment or leading to next discussion


# Setting Up Docker

* Easy Peasy, look for the steps for your host OS at Docker official page. In general, we can rely on yum/apt for Linux system.

  [Install Docker for Ubuntu](https://docs.docker.com/install/linux/docker-ce/ubuntu/)



* Slightly complicated if you are behind proxy, just like Intel Network.

  [BKM on Intel Network](https://gitlab.devtools.intel.com/vchua/docker-setup)
  
  Please share your BKM too!

# Special Config for Today (1/2)

* Docker requires root access to operate. Do the following to avoid putting ```sudo``` for every docker command.

   ```sudo groupadd docker```
   
   ```sudo gpasswd -a $USER docker```


exit your shell, and re-log in


* Add the following to your ```~/.bashrc```
```bash
export DOCKER_PROXY_RUN_ARGS="\
            --env HTTPS_PROXY=$HTTPS_PROXY \
            --env https_proxy=$https_proxy \
            --env HTTP_PROXY=$HTTP_PROXY \
            --env http_proxy=$http_proxy \
            --env NO_PROXY=$NO_PROXY \
            --env no_proxy=$no_proxy \
            --dns 10.248.2.1"
```
```bash
export DOCKER_PROXY_BUILD_ARGS="\
            --build-arg HTTPS_PROXY=$HTTPS_PROXY \
            --build-arg https_proxy=$https_proxy \
            --build-arg HTTP_PROXY=$HTTP_PROXY \
            --build-arg http_proxy=$http_proxy \
            --build-arg NO_PROXY=$NO_PROXY \
            --build-arg no_proxy=$no_proxy"
```

# Special Config for Today (2/2)

* If you want to access internet in the container runtime, you need to supply the proxy config during instantiation

  ```source ~/.bashrc``` if you do not re-log in.

  ```docker run $DOCKER_PROXY_RUN_ARGS ubuntu```
  
  
*These two slides are specific to the training today. Will be removed later*

# Survival Guide

```docker help```

This help provides a short description for each of the docker command. For extensive help, search online or go to https://docs.docker.com/reference/  

.................................................................................................................................................................................

```docker <command> --help```

There are switches w.r.t the docker commands. Use ```--help``` to get the description for the switches


##### Quiz: how many switches are there for docker ```stop``` command?

# Hello World

```docker run hello-world```

```
vchua@flexpg-vbox1:~$ docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
1b930d010525: Pull complete
Digest: sha256:b8ba256769a0ac28dd126d584e0a2011cd2877f3f76e093a7ae560f2a5301c00
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/
```

# Where does the image come from?

### [Docker Hub](https://hub.docker.com/)

```search with keyword hello-world```

##### Quiz: what information is available?
##### Quiz: Try to search some popular apps?

## *Pretty much you can run anything that somebody has shared. We don't have to rebuild!*

___________
# Run a ubuntu image
```docker run ubuntu```

##### Quiz: Discuss what you see in the output?

```
vchua@flexpg-vbox1:~$ docker run ubuntu

Unable to find image 'ubuntu:latest' locally
latest: Pulling from library/ubuntu
5667fdb72017: Pull complete
d83811f270d5: Pull complete
ee671aafb583: Pull complete
7fc152dfb3a6: Pull complete
Digest: sha256:b88f8848e9a1a4e4558ba7cfc4acc5879e1d0e7ac06401409062ad2627e6fb58
Status: Downloaded newer image for ubuntu:latest
vchua@flexpg-vbox1:~$
```

> *NOTHING HAPPEN after image is downloaded?!*

##### Quiz: Why?

# List container images in your system

```docker images```

```
vchua@flexpg-vbox1:~$ docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
ubuntu              latest              2ca708c1c9cc        2 weeks ago         64.2MB
hello-world         latest              fce289e99eb9        9 months ago        1.84kB
```

##### Quiz: What is TAG?

# Container Tag

Let's download a few ubuntu version to our system.

```docker pull ubuntu:16.04```

```docker pull ubuntu:18.04```

```
vchua@flexpg-vbox1:~$ docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
ubuntu              16.04               657d80a6401d        2 weeks ago         121MB
ubuntu              18.04               2ca708c1c9cc        2 weeks ago         64.2MB
ubuntu              latest              2ca708c1c9cc        2 weeks ago         64.2MB
hello-world         latest              fce289e99eb9        9 months ago        1.84kB
```

Observe the Image Id, do you see the Id of ubuntu:latest == Id of ubuntu:18.04?

##### Quiz: What is purpose of tag? how can it be used for?

# Tag an image

```docker tag ubuntu:16.04 favorite-ubt:v0```

```
vchua@flexpg-vbox1:~$ docker tag ubuntu:16.04 favorite-ubt:v0
vchua@flexpg-vbox1:~$ docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
favorite-ubt        v0                  657d80a6401d        2 weeks ago         121MB
ubuntu              16.04               657d80a6401d        2 weeks ago         121MB
ubuntu              18.04               2ca708c1c9cc        2 weeks ago         64.2MB
ubuntu              latest              2ca708c1c9cc        2 weeks ago         64.2MB
hello-world         latest              fce289e99eb9        9 months ago        1.84kB
```

[More on tag](https://docs.docker.com/engine/reference/commandline/tag/)

# Entrypoint

Entrypoint is the default command that is executed when you instantiate a container. For example, the ```hello-world``` executes the program to print "Hello from Docker" and the rest of the information. The instance exits once the entrypoint command is complete.

There are images without an entrypoint. In that case, nothing will happen, just like you did ```docker run ubuntu``` earlier.

##### Quiz: How can i know the entrypoint of a container?


--------------------

# Override Entrypoint
```docker run ubuntu echo "Great to see you again, pal!"```

```
vchua@flexpg-vbox1:~$ docker run ubuntu echo "Great to see you again, pal!"
Great to see you again, pal!
```



# Interactive Shell in Ubuntu instance

Now, we can override the entry command, since it is a ubuntu image, it will be useful we can use it like a native OS.

```docker run $DOCKER_PROXY_RUN_ARGS -i -t ubuntu bash```

```
vchua@flexpg-vbox1:~$ docker run -i -t ubuntu:18.04 bash
root@3f447cf241e8:/#
root@3f447cf241e8:/# cat /etc/lsb-release
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=18.04
DISTRIB_CODENAME=bionic
DISTRIB_DESCRIPTION="Ubuntu 18.04.3 LTS"
```
Viola! See that? It is a ubuntu 18.04, where as your host is 16.04.

##### Quiz: what do ```-i``` and ```-t``` do?


Now, try apt install some packages.
```
root@3f447cf241e8:/# python --version
bash: python: command not found
```
I suggest python since this image doesn't come with python

```apt-get update && apt-get install python```

```
root@3f447cf241e8:/# python --version
Python 2.7.15+
```

# Check Container Instances

Do not exit the interactive shell earlier. Open another terminal, log in to your system and do the following

```docker ps```

```
vchua@flexpg-vbox1:~$ docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
3f447cf241e8        ubuntu:18.04        "/bin/bash"         53 minutes ago      Up 53 minutes                           priceless_ardinghelli
```

This is analogous to the linux ```ps``` command, for docker, it lists the instantiated containers, by default, it returns the containers that are alive.

##### Quiz: Find out how to list the exited containers, how to filter the ```docker ps``` table

# More Shell with the same instance


```
vchua@flexpg-vbox1:~$ docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
3f447cf241e8        ubuntu:18.04        "/bin/bash"         53 minutes ago      Up 53 minutes                           priceless_ardinghelli
```

Try the following:

```docker exec -i -t priceless_ardinghelli bash```

```docker exec -i -t 3f447cf241e8 bash```

```docker exec priceless_ardinghelli python --version```

# Commit Changes done in Interactive Session

##### Quiz: what happen if we exit the shell now?

When we exit the interactive shell, we lose all the work we've done during session. To save the work, docker provides a ```commit``` command that create another container image that preserves the work has been performed.

Using the instance that has python installed, let's commit it to an image. 

To do that, we need to get the **Container Id** or **Name** of the instance, open another terminal and log in to your system.

```docker ps```
```
vchua@flexpg-vbox1:~$ docker ps
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
3f447cf241e8        ubuntu:18.04        "/bin/bash"         53 minutes ago      Up 53 minutes                           priceless_ardinghelli
```

```docker commit 3f447cf241e8 ubt1804:py3``` or 

```docker commit priceless_ardinghelli ubt1804:py3```

```
vchua@flexpg-vbox1:~$ docker commit 3f447cf241e8 ubt1804:py3
sha256:a2193a2181e603b18536240678fd0f57a40aea41755dc36817ebc0c9af10cdc9
```

List the local container images and check if the new image exists.

```docker images```

```
vchua@flexpg-vbox1:~$ docker images
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
ubt1804             py3                 a2193a2181e6        3 minutes ago       491MB
ubuntu              16.04               657d80a6401d        2 weeks ago         121MB
favorite-ubt        v0                  657d80a6401d        2 weeks ago         121MB
ubuntu              18.04               2ca708c1c9cc        2 weeks ago         64.2MB
ubuntu              latest              2ca708c1c9cc        2 weeks ago         64.2MB
hello-world         latest              fce289e99eb9        9 months ago        1.84kB
```


Now, let's see if python is in the new image.

```docker run ubt1804:py3 python --version```
```
vchua@flexpg-vbox1:~$ docker run ubt1804:py3 python --version
Python 2.7.15+
```

# Share Container Images

Thus far, when we do ```docker images```, we see the containers which are available locally, the downloaded images and/or the ones we build. Yes, you can come back the other day and start from your committed container but this isn't great enough, your great apps should be shared or deployed. How can we transfer the container to an external system?

1. Native docker ```save``` and ```load``` commands
2. Using a registry
    - Docker Hub
    - Registry hosted by anyone

# Save, Share & Load

Let's share the container ```ubt1804:py3```

```docker save ubt1804:py3 -o ubt1804-py3.tar.gz```

We can share ubt1804-py3.tar via any methods for any file, i.e. cp, scp, thumbdrive etc

To recover the tarball at an external system,

```docker load -i ubt1804-py3.tar.gz```

Do ```docker images``` once the above is complete

# Using a Registry - Docker Hub

Registry is the appropriate way to share image, we can manage the access of the images. The steps to publish or push an image:
1. log in to registry. When no registry address is provided, this logs in to dockerhub.

```docker login```
   
2. tag the image with account as prefix of the image tag. 

```docker tag ubt1804:py3 vuiseng9/ubt1804:py3```

3. push the newly-tagged image to registry

```docker push vuiseng9/ubt1804:py3```

Following is how I share the image publicly to dockerhub account
```
vchua@flexpg-vbox1:~$ docker login
Login with your Docker ID to push and pull images from Docker Hub. If you don't have a Docker ID, head over to https://hub.docker.com to create one.
Username: vuiseng9
Password:
WARNING! Your password will be stored unencrypted in /home/vchua/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


vchua@flexpg-vbox1:~$ docker tag ubt1804:py3 vuiseng9/ubt1804:py3

vchua@flexpg-vbox1:~$ docker images
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
ubt1804             py3                 a2193a2181e6        3 hours ago         491MB
vuiseng9/ubt1804    py3                 a2193a2181e6        3 hours ago         491MB
favorite-ubt        v0                  657d80a6401d        2 weeks ago         121MB
ubuntu              16.04               657d80a6401d        2 weeks ago         121MB
ubuntu              18.04               2ca708c1c9cc        2 weeks ago         64.2MB
ubuntu              latest              2ca708c1c9cc        2 weeks ago         64.2MB
hello-world         latest              fce289e99eb9        9 months ago        1.84kB


vchua@flexpg-vbox1:~$ docker push vuiseng9/ubt1804:py3
The push refers to repository [docker.io/vuiseng9/ubt1804]
a61f6cdb0908: Pushed
e80c789bc6ac: Mounted from library/ubuntu
6c3332381368: Mounted from library/ubuntu
ef1a1ec5bba9: Mounted from library/ubuntu
a1aa3da2a80a: Mounted from library/ubuntu
```


# Using Container Registry of Intel CaaS

[Intel Container as a Service](https://soco.intel.com/groups/caas-evaluation-workgroup/blog/2018/11/06/introducing-containers-as-a-service-in-production) provides container registry service for convenient image sharing internally. It uses [Harbor](https://goharbor.io/) which integrates well with Intel AGS process and can easily control the access level of the shared containers.

Good News! We have created a project for our day-to-day usage and your access has been approved! Let's go for a spin.

**Browse https://gar-registry.caas.intel.com and navigate to project flexpg and see the available images**


To interact (push, pull etc) with CaaS Registry, we need to download the CA certifate.  
```bash
git clone https://gitlab.devtools.intel.com/caas-public/general && chmod +x general/scripts/ca_install.sh && sudo general/scripts/ca_install.sh && rm -rf general

sudo systemctl restart docker
```

Back to the topic, let's share our container internally.



1. log in to registry with your windows credential

```docker login https://gar-registry.caas.intel.com```

2. tag the image with registry and project as prefix of the image tag. 
  
```docker tag ubt1804:py3 gar-registry.caas.intel.com/flexpg/ubt1804:py3```

3. push the newly-tagged image to registry

```docker push gar-registry.caas.intel.com/flexpg/ubt1804:py3```


##### Quiz: Use Rest API to list the images in the registrt
##### Quiz: Try pulling image from CaaS

```
vchua@flexpg-vbox1:~$ docker login https://gar-registry.caas.intel.com
Username: vchua
Password:
WARNING! Your password will be stored unencrypted in /home/vchua/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


vchua@flexpg-vbox1:~$ docker tag ubt1804:py3 gar-registry.caas.intel.com/flexpg/ubt1804:py3


vchua@flexpg-vbox1:~$ docker images
REPOSITORY                                   TAG                 IMAGE ID            CREATED             SIZE
vuiseng9/ubt1804                             py3                 a2193a2181e6        12 hours ago        491MB
gar-registry.caas.intel.com/flexpg/ubt1804   py3                 a2193a2181e6        12 hours ago        491MB
ubt1804                                      py3                 a2193a2181e6        12 hours ago        491MB
ubuntu                                       16.04               657d80a6401d        2 weeks ago         121MB
favorite-ubt                                 v0                  657d80a6401d        2 weeks ago         121MB
ubuntu                                       18.04               2ca708c1c9cc        2 weeks ago         64.2MB
ubuntu                                       latest              2ca708c1c9cc        2 weeks ago         64.2MB
hello-world                                  latest              fce289e99eb9        9 months ago        1.84kB


vchua@flexpg-vbox1:~$ docker push gar-registry.caas.intel.com/flexpg/ubt1804:py3
The push refers to repository [gar-registry.caas.intel.com/flexpg/ubt1804]
a61f6cdb0908: Pushed
e80c789bc6ac: Pushed
6c3332381368: Pushed
ef1a1ec5bba9: Pushed
a1aa3da2a80a: Pushed
py3: digest: sha256:fc3deecd75b7a525d692ca863dc5c7b1ab0fe907f2e368d46fd19685829bab2c size: 1365

```

# Assignment #1

1. Containerize your favorite app through the interactive docker instance.
2. Push your image to our registry at **gar-registry.caas.intel.com/flexpg**
3. Email to vui.seng.chua@intel.com with a short description of your container and the docker command with entrypoint
4. [Optional] Create an account in dockerhub and share your app to the world! When you do this, please beware of sharing Intel restricted IP and information!

# Next Session

Focus on 

## Automated Container Build - Dockerfile
